In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def detect_paper(frame):

    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(frame,(3,3),0)

    thresh = cv2.threshold(blur,254,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

    kernel = np.ones((7,7),np.uint8)
    morph = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE, kernel=kernel)
    morph = cv2.morphologyEx(morph,cv2.MORPH_OPEN, kernel=kernel)

    contours = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contours = contours[0] if len(contours) == 2 else contours[1]
    area_thresh = 0
    for c in contours:
        area = cv2.contourArea(c)
        if area > area_thresh:
            area_thresh = area
            contourMax = c

    #contourMax = max(contours, key=cv2.contourArea)

    paper = np.zeros_like(img)

    cv2.drawContours(paper,[contourMax], 0, (255,255,255),-1)

    peri = cv2.arcLength(contourMax,True)
    corner = cv2.approxPolyDP(contourMax,0.02*peri, True)

    if len(corner) == 4:
        cv2.drawContours(frame,[corner],0,(0,255,0),2)

    return frame

In [4]:
alive = True

win_name = "Camera Filters"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
result = None
source = cv2.VideoCapture(0)

while alive:
    has_frame, frame = source.read()
    if not has_frame:
        break

    frame = detect_paper(frame)

    cv2.imshow(win_name,frame)

    key = cv2.waitKey(1)
    if key == ord("Q") or key == ord("q") or key == 27:
        alive = False